### Imports:

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [3]:
# for tcn applications
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, RepeatVector
from tensorflow.keras.preprocessing import sequence
from tcn import TCN, compiled_tcn
import tensorflow as tf
import os
os.chdir("/Users/ridhaalkhabaz/Documents/GitHub/ElectricDemandForecasting-DL/src/")
from models import tcn
# for data configurations
import sys
import zipfile
from dateutil.parser import parse
import json
from random import shuffle
import random
import datetime
import math
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot, dates
from datetime import timedelta
from pandas import read_csv
from pandas import datetime
from numpy import array
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import datetime as dttm

### Data configuration and Reading:

#### First: Riser data:

In [4]:
# Convert the sequence of seconds into datetime array
def parser(x):
    now_seconds = 0
    y = x.astype(np.float) + now_seconds
    z = pd.to_datetime(y, unit='s')
    return z

In [31]:
# Read the data from fname and eventually plot them
def read_data(fname, plot_data = False):
    # Read the time series
    datats = read_csv(fname, header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)

    headers = list(datats.columns.values)
    headers.insert(0, datats.index.name)

    # Resample the data using a uniform timestep
    datats = datats.resample('S').mean()
    datats = datats.interpolate(method='linear')
    # Convert data to numpy array
    data = datats.reset_index().values

    # Replace timestamps with seconds
    time_sec = array([data[i, 0].timestamp() for i in range(len(data))])
    data = np.c_[time_sec, data[:, 1:]]

    # Plot the pressure readings
    if (plot_data):
        pyplot.plot(data[:, 0], data[:, 1:8])
        pyplot.xlabel(headers[0])
        pyplot.ylabel('Pressure (bar)')
        # Use the original headers
        # headersplot = [w.replace('x_', '$x_') for w in headers[1:8]]
        # headersplot = [w.replace('}=', '}$=') for w in headersplot]
        # headersplot = [w.replace(' (bar)', '') for w in headersplot]
        # Use the headers p(x=xi)
        headersplot = [w[-8:-2] for w in headers[1:8]]
        px = ['$p(x_{%d}' % i for i in range(1, 8)]
        tail = [')$'] * 7
        headersplot = [px + headersplot + tail for px, headersplot, tail in zip(px, headersplot, tail)]
        pyplot.legend(headersplot)
        pyplot.title('Distributed pressure readings')
        pyplot.show(block=False)
        pyplot.savefig('pressure_readings.pdf')
    # some outliers and spikes
#     for i in random.sample(range(1, 3000), 100):
#         data[i][1:] = random.sample(range(1, 3000), 9)
        
    # Normalize the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(data)
    scaler.scale_[0] = 1    # Do not normalize time

    # Apply the same normalization to all pressure readings
    pind = list(range(1, 8))  # Indices of pressure readings
    pmin = scaler.data_min_[pind].min()
    pmax = scaler.data_max_[pind].max()
    scaler.scale_[pind] = ((scaler.feature_range[1] - scaler.feature_range[0]) / (pmax - pmin))
    scaler.min_[pind] = scaler.feature_range[0] - pmin * scaler.scale_[pind]

    data = scaler.transform(data)

    return data, scaler

In [32]:
os.chdir("/Users/ridhaalkhabaz/Documents/GitHub/DeepAR_Prophet_VFM")

In [33]:
data, scalar = read_data('riser_pq_uni.csv')

In [34]:
def get_xy_kfolds(split_index=[0.5, 0.6, 0.7, 0.8, 0.9], timesteps=1000):
    """
    load exchange rate dataset and preprecess it, then split it into k-folds for CV
    :param split_index: list, the ratio of whole dataset as train set
    :param timesteps: length of a single train x sample
    :return: list, [train_x_set,train_y_set,test_x_single,test_y_single]
    """
    df = data
    n = len(df)
    folds = []
    for split_point in split_index:
        train_end = int(split_point * n)
        train_x, train_y = [], []
        for i in range(train_end - timesteps):
            train_x.append(df[i:i + timesteps, 0:8])
            train_y.append(df[i + timesteps, 8:10])
        train_x = np.array(train_x)
        train_y = np.array(train_y)
        test_x = df[train_end - timesteps + 1:train_end + 1, 0:8]
        test_y = df[train_end + 1,  8:10]
        folds.append((train_x, train_y, test_x, test_y))
    return folds

In [35]:
folds = get_xy_kfolds()
dft = pd.DataFrame.from_dict(dict(zip(['ds', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7','gas(m3/day)', 'liquid(m3/day)' ], data.T)))


,ds,p1,p2,p3,p4,p5,p6,p7,gas(m3/day),liquid(m3/day)
0,0.0,0.904939,0.881934,0.721893,0.526299,0.337001,0.155666,0.016581,0.210728,0.365316
1,1.0,0.907502,0.889475,0.732448,0.536326,0.346176,0.163545,0.025615,0.192040,0.289380
2,2.0,0.903671,0.883730,0.728871,0.532214,0.341866,0.159461,0.020827,0.219544,0.354832
3,3.0,0.900143,0.880012,0.727829,0.531686,0.341796,0.159701,0.021207,0.237477,0.382744
4,4.0,0.897756,0.877927,0.727519,0.531482,0.341613,0.159602,0.021137,0.247850,0.398670


#### Training of our proposed model:

In [ ]:
mse_list = []
for train_x,train_y,test_x,test_y in folds:
    model = compiled_tcn(return_sequences=False,
                         num_feat=test_x.shape[1],
                         nb_filters=24,
                         num_classes=0,
                         kernel_size=8,
                         dilations=[2 ** i for i in range(9)],
                         nb_stacks=2,
                         max_len=test_x.shape[0],
                         use_skip_connections=True,
                         regression=True,
                         dropout_rate=0,
                         output_len= test_y.shape[0])
    model.fit(train_x,train_y,batch_size=256,epochs=100)
    y_pred = model.predict(np.array([test_x]))
    mse_cur = mean_squared_error(y_pred[0],test_y)
    mse_list.append(mse_cur)
    print(f"train_set_size:{train_x.shape[0]}")
    print(f"y_true:{test_y}")
    print(f"y_pred:{y_pred}")
    print(f"mse:{mse_cur}")
print(np.mean(mse_list))

x.shape= (None, 24)
model.x = (None, 1000, 8)
model.y = (None, 2)
Epoch 1/100
2/2 [==============================] - 22s 6s/step - loss: 11754621610.6667
Epoch 2/100
2/2 [==============================] - 13s 6s/step - loss: 162524533.3333
Epoch 3/100
2/2 [==============================] - 13s 6s/step - loss: 22365754.0000
Epoch 4/100
2/2 [==============================] - 13s 6s/step - loss: 8503057.0000
Epoch 5/100
2/2 [==============================] - 13s 6s/step - loss: 3104427.7708
Epoch 6/100
2/2 [==============================] - 17s 9s/step - loss: 1868433.2500
Epoch 7/100
2/2 [==============================] - 20s 9s/step - loss: 531550.8958
Epoch 8/100
2/2 [==============================] - 18s 8s/step - loss: 795471.0000
Epoch 9/100
2/2 [==============================] - 16s 7s/step - loss: 830503.9167
Epoch 10/100
2/2 [==============================] - 15s 7s/step - loss: 198473.3464
Epoch 11/100
2/2 [==============================] - 18s 9s/step - loss: 145824.6979
Epoch 

2/2 [==============================] - 14s 6s/step - loss: 1163.3998
Epoch 100/100
2/2 [==============================] - 13s 7s/step - loss: 1321.1658
train_set_size:500
y_true:[0.28182131 0.45939675]
y_pred:[[-62.57203   48.715874]]
mse:3139.647041023783
x.shape= (None, 24)
model.x = (None, 1000, 8)
model.y = (None, 2)
Epoch 1/100
4/4 [==============================] - 30s 5s/step - loss: 410000228352.0000
Epoch 2/100
4/4 [==============================] - 22s 5s/step - loss: 250681932.8000
Epoch 3/100
4/4 [==============================] - 22s 5s/step - loss: 17422346.8000
Epoch 4/100
4/4 [==============================] - 23s 5s/step - loss: 3022572.0500
Epoch 5/100
4/4 [==============================] - 22s 5s/step - loss: 1143147.1750
Epoch 6/100
4/4 [==============================] - 22s 5s/step - loss: 332787.7313
Epoch 7/100
4/4 [==============================] - 22s 5s/step - loss: 979706.7125
Epoch 8/100
4/4 [==============================] - 23s 5s/step - loss: 503302.1937


4/4 [==============================] - 23s 5s/step - loss: 10850.9582
Epoch 88/100
4/4 [==============================] - 23s 5s/step - loss: 5197.8253
Epoch 89/100
4/4 [==============================] - 23s 5s/step - loss: 8155.9896
Epoch 90/100
4/4 [==============================] - 23s 5s/step - loss: 4797.6690
Epoch 91/100
4/4 [==============================] - 22s 5s/step - loss: 4056.2073
Epoch 92/100
4/4 [==============================] - 23s 5s/step - loss: 987.5132
Epoch 93/100
4/4 [==============================] - 32s 7s/step - loss: 1355.6452
Epoch 94/100
4/4 [==============================] - 28s 6s/step - loss: 1661.3464
Epoch 95/100
4/4 [==============================] - 28s 6s/step - loss: 651.1589
Epoch 96/100
4/4 [==============================] - 28s 6s/step - loss: 2173.1430
Epoch 97/100
4/4 [==============================] - 27s 6s/step - loss: 1763.1556
Epoch 98/100
4/4 [==============================] - 27s 6s/step - loss: 2325.7076
Epoch 99/100
4/4 [============

5/5 [==============================] - 28s 5s/step - loss: 3327.6920
Epoch 76/100
5/5 [==============================] - 28s 5s/step - loss: 2569.0852
Epoch 77/100
5/5 [==============================] - 29s 6s/step - loss: 2014.8697
Epoch 78/100
5/5 [==============================] - 29s 5s/step - loss: 770.0214
Epoch 79/100
5/5 [==============================] - 28s 5s/step - loss: 888.6344
Epoch 80/100
5/5 [==============================] - 28s 5s/step - loss: 3409.1374
Epoch 81/100
5/5 [==============================] - 29s 5s/step - loss: 1661.6482
Epoch 82/100
5/5 [==============================] - 29s 5s/step - loss: 1292.6334
Epoch 83/100
5/5 [==============================] - 29s 6s/step - loss: 1822.9425
Epoch 84/100
5/5 [==============================] - 28s 5s/step - loss: 963.0994
Epoch 85/100
5/5 [==============================] - 28s 5s/step - loss: 870.2969
Epoch 86/100
5/5 [==============================] - 28s 5s/step - loss: 876.6888
Epoch 87/100
5/5 [================

6/6 [==============================] - 66s 11s/step - loss: 1233.0403
Epoch 64/100
6/6 [==============================] - 71s 11s/step - loss: 4381.2580
Epoch 65/100
6/6 [==============================] - 67s 11s/step - loss: 1495.9540
Epoch 66/100
6/6 [==============================] - 68s 11s/step - loss: 1041.2562
Epoch 67/100
6/6 [==============================] - 66s 11s/step - loss: 367.6181
Epoch 68/100
6/6 [==============================] - 67s 11s/step - loss: 2290.0913
Epoch 69/100
6/6 [==============================] - 66s 11s/step - loss: 2716.6686
Epoch 70/100
6/6 [==============================] - 71s 12s/step - loss: 1187.8560
Epoch 71/100
6/6 [==============================] - 70s 12s/step - loss: 885.1183
Epoch 72/100
6/6 [==============================] - 68s 11s/step - loss: 145.7277
Epoch 73/100
6/6 [==============================] - 69s 11s/step - loss: 364.0473
Epoch 74/100
6/6 [==============================] - 69s 11s/step - loss: 665.1840
Epoch 75/100
6/6 [====

In [47]:
y_pred = model.predict(np.array([test_x]))
mse_cur = mean_squared_error(y_pred[0],test_y)

In [48]:
mse_cur

265.6242636691044

In [204]:

# batch_size, timesteps, input_dim = 64, 23, 8
# output_timesteps, output_dim = 1, 2

# model = Sequential(
#     layers=[
#         TCN(input_shape=(timesteps, input_dim)),  # output.shape = (batch, 64)
#         RepeatVector(output_timesteps),  # output.shape = (batch, output_timesteps, 64)
#         Dense(output_dim)  # output.shape = (batch, output_timesteps, output_dim)
#     ]
# )

# model.summary()
# model.compile('adam', 'mse')

# print('Train...')
# model.fit(batch_x, batch_y,epochs=1000, batch_size=batch_size)


Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_4 (Functional)         (None, 30)                111872    
_________________________________________________________________
repeat_vector_15 (RepeatVect (None, 1, 30)             0         
_________________________________________________________________
dense_26 (Dense)             (None, 1, 2)              62        
Total params: 111,934
Trainable params: 111,934
Non-trainable params: 0
_________________________________________________________________
Train...
Epoch 1/500
1/1 [==============================] - 2s 2s/step - loss: 106923458560.0000
Epoch 2/500
1/1 [==============================] - 0s 76ms/step - loss: 117198315520.0000
Epoch 3/500
1/1 [==============================] - 0s 77ms/step - loss: 48350212096.0000
Epoch 4/500
1/1 [==============================] - 0s 80ms/step - loss: 16601337856.0000
Epoch 5/500


Epoch 85/500
1/1 [==============================] - 0s 97ms/step - loss: 541900544.0000
Epoch 86/500
1/1 [==============================] - 0s 97ms/step - loss: 456348480.0000
Epoch 87/500
1/1 [==============================] - 0s 74ms/step - loss: 541441472.0000
Epoch 88/500
1/1 [==============================] - 0s 76ms/step - loss: 530690784.0000
Epoch 89/500
1/1 [==============================] - 0s 87ms/step - loss: 384694080.0000
Epoch 90/500
1/1 [==============================] - 0s 89ms/step - loss: 296293184.0000
Epoch 91/500
1/1 [==============================] - 0s 88ms/step - loss: 351931200.0000
Epoch 92/500
1/1 [==============================] - 0s 89ms/step - loss: 329549152.0000
Epoch 93/500
1/1 [==============================] - 0s 93ms/step - loss: 419971936.0000
Epoch 94/500
1/1 [==============================] - 0s 99ms/step - loss: 363620928.0000
Epoch 95/500
1/1 [==============================] - 0s 94ms/step - loss: 433811808.0000
Epoch 96/500
1/1 [==============

1/1 [==============================] - 0s 83ms/step - loss: 248384240.0000
Epoch 178/500
1/1 [==============================] - 0s 87ms/step - loss: 147700208.0000
Epoch 179/500
1/1 [==============================] - 0s 83ms/step - loss: 234852528.0000
Epoch 180/500
1/1 [==============================] - 0s 83ms/step - loss: 218874272.0000
Epoch 181/500
1/1 [==============================] - 0s 80ms/step - loss: 232102624.0000
Epoch 182/500
1/1 [==============================] - 0s 78ms/step - loss: 229324528.0000
Epoch 183/500
1/1 [==============================] - 0s 89ms/step - loss: 293745280.0000
Epoch 184/500
1/1 [==============================] - 0s 86ms/step - loss: 177235616.0000
Epoch 185/500
1/1 [==============================] - 0s 82ms/step - loss: 183032160.0000
Epoch 186/500
1/1 [==============================] - 0s 84ms/step - loss: 162333792.0000
Epoch 187/500
1/1 [==============================] - 0s 83ms/step - loss: 236883424.0000
Epoch 188/500
1/1 [================

1/1 [==============================] - 0s 92ms/step - loss: 124396632.0000
Epoch 270/500
1/1 [==============================] - 0s 90ms/step - loss: 155411136.0000
Epoch 271/500
1/1 [==============================] - 0s 89ms/step - loss: 123963144.0000
Epoch 272/500
1/1 [==============================] - 0s 89ms/step - loss: 134690064.0000
Epoch 273/500
1/1 [==============================] - 0s 89ms/step - loss: 118251472.0000
Epoch 274/500
1/1 [==============================] - 0s 90ms/step - loss: 189697600.0000
Epoch 275/500
1/1 [==============================] - 0s 90ms/step - loss: 99306920.0000
Epoch 276/500
1/1 [==============================] - 0s 82ms/step - loss: 87127984.0000
Epoch 277/500
1/1 [==============================] - 0s 80ms/step - loss: 172443184.0000
Epoch 278/500
1/1 [==============================] - 0s 74ms/step - loss: 122104448.0000
Epoch 279/500
1/1 [==============================] - 0s 73ms/step - loss: 156639232.0000
Epoch 280/500
1/1 [==================

1/1 [==============================] - 0s 75ms/step - loss: 111604488.0000
Epoch 362/500
1/1 [==============================] - 0s 70ms/step - loss: 115771232.0000
Epoch 363/500
1/1 [==============================] - 0s 71ms/step - loss: 88728960.0000
Epoch 364/500
1/1 [==============================] - 0s 69ms/step - loss: 86934912.0000
Epoch 365/500
1/1 [==============================] - 0s 70ms/step - loss: 108048096.0000
Epoch 366/500
1/1 [==============================] - 0s 70ms/step - loss: 97370136.0000
Epoch 367/500
1/1 [==============================] - 0s 71ms/step - loss: 115907216.0000
Epoch 368/500
1/1 [==============================] - 0s 68ms/step - loss: 66260516.0000
Epoch 369/500
1/1 [==============================] - 0s 80ms/step - loss: 70930704.0000
Epoch 370/500
1/1 [==============================] - 0s 70ms/step - loss: 98588840.0000
Epoch 371/500
1/1 [==============================] - 0s 74ms/step - loss: 91966280.0000
Epoch 372/500
1/1 [=======================

1/1 [==============================] - 0s 94ms/step - loss: 63828928.0000
Epoch 455/500
1/1 [==============================] - 0s 94ms/step - loss: 62699576.0000
Epoch 456/500
1/1 [==============================] - 0s 90ms/step - loss: 85479560.0000
Epoch 457/500
1/1 [==============================] - 0s 91ms/step - loss: 69026544.0000
Epoch 458/500
1/1 [==============================] - 0s 89ms/step - loss: 67321528.0000
Epoch 459/500
1/1 [==============================] - 0s 89ms/step - loss: 92090152.0000
Epoch 460/500
1/1 [==============================] - 0s 91ms/step - loss: 84477584.0000
Epoch 461/500
1/1 [==============================] - 0s 88ms/step - loss: 61461176.0000
Epoch 462/500
1/1 [==============================] - 0s 86ms/step - loss: 92946624.0000
Epoch 463/500
1/1 [==============================] - 0s 90ms/step - loss: 67856568.0000
Epoch 464/500
1/1 [==============================] - 0s 89ms/step - loss: 98583128.0000
Epoch 465/500
1/1 [===========================

In [174]:
# FORECAST_HORIZON = 30
# PAST_HISTORY = 40
# BATCH_SIZE = 64
# EPOCHS = 30
# METRICS = ['mse']

# TCN_PARAMS = {
#     'nb_filters': 128,
#     'kernel_size': 4,
#     'nb_stacks': 1,
#     'dilations': [1, 2, 4, 8, 16, 32, 64],
#     'dropout_rate': 0,
# }

In [175]:
# model = tcn(batch_x.shape, FORECAST_HORIZON, 'adam', 'mse', 
#             nb_filters=TCN_PARAMS['nb_filters'],
#             kernel_size=TCN_PARAMS['kernel_size'],
#             nb_stacks= TCN_PARAMS['nb_stacks'],
#             dilations=TCN_PARAMS['dilations'],
#             dropout_rate=TCN_PARAMS['dropout_rate'])
# model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 23, 8)]           0         
_________________________________________________________________
tcn_12 (TCN)                 (None, 128)               859008    
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 30)                3870      
_________________________________________________________________
dense_13 (Dense)             (None, 30)                930       
Total params: 863,808
Trainable params: 863,808
Non-trainable params: 0
_________________________________________________________________


In [184]:
# # Convert numpy data to tensorflow dataset
# train_data = tf.data.Dataset.from_tensor_slices((batch_x, batch_y)).cache().batch(BATCH_SIZE)
# val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(BATCH_SIZE).repeat()
# test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

In [185]:
# model.compile(
#     optimizer='adam', loss='mse', metrics=None, loss_weights=None,
#     weighted_metrics=None, run_eagerly=None, steps_per_execution=None)

In [189]:
# train_data

<BatchDataset shapes: ((None, 23, 8), (None, 23, 2)), types: (tf.float64, tf.float64)>

In [192]:
# TRAIN_MODEL = True   

# checkpoint_path = "training_tcn/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# if TRAIN_MODEL:
#     evaluation_interval = 100
#     history = model.fit(train_data, 
#                         epochs=EPOCHS,
#                         steps_per_epoch=evaluation_interval,
#                         validation_data=val_data, validation_steps=evaluation_interval)

Epoch 1/30


ValueError: in user code:

    /Users/ridhaalkhabaz/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/ridhaalkhabaz/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/ridhaalkhabaz/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/ridhaalkhabaz/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/ridhaalkhabaz/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /Use

In [76]:
# batch_size, timesteps, input_dim = 64, 24, 8
# output_timesteps, output_dim = 6, 2

# # dummy values here. There is nothing to learn. It's just to show how to do it.
# batch_x = np.random.uniform(size=(batch_size, timesteps, input_dim))
# batch_y = np.random.uniform(size=(batch_size, output_timesteps, output_dim))

# model = Sequential(
#     layers=[
#         TCN(input_shape=(timesteps, input_dim)),  # output.shape = (batch, 64)
#         RepeatVector(output_timesteps),  # output.shape = (batch, output_timesteps, 64)
#         Dense(output_dim)  # output.shape = (batch, output_timesteps, output_dim)
#     ]
# )

# model.summary()
# model.compile('adam', 'mse')

# print('Train...')
# model.fit(batch_x, batch_y, batch_size=batch_size)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tcn_3 (TCN)                  (None, 64)                138048    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 6, 64)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 6, 2)              130       
Total params: 138,178
Trainable params: 138,178
Non-trainable params: 0
_________________________________________________________________
Train...
1/1 [==============================] - 2s 2s/step - loss: 33.1563
